# 1-rdt-feature-testing

In [1]:
import pandas as pd
import catboost
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

testing which version of year will result in better model performance

In [2]:
# set seed
seed = 42

In [3]:
# import data
df = pd.read_csv("../data/interim/corpus_studio_pre_processed.csv")
df = df.fillna("")
df.isna().sum()
df.head()

,word,previous_word,next_word,artist,song_type,song,aae_feature,aae_realization,performance_type,time,social_group,year
0,there,,is,albertking,cover,as_the_years_go_passing_by,post-vocalic r,0,studio,1960s,AA,1969
1,there,,is,albertking,cover,as_the_years_go_passing_by,post-vocalic r,0,studio,1960s,AA,1967
2,is,there,nothing,albertking,cover,as_the_years_go_passing_by,auxiliary verb,0,studio,1960s,AA,1969
3,is,there,nothing,albertking,cover,as_the_years_go_passing_by,auxiliary verb,0,studio,1960s,AA,1967
4,nothing,is,I,albertking,cover,as_the_years_go_passing_by,ing ultimas,1,studio,1960s,AA,1969


In [4]:
# drop redundant column
df.drop(columns="performance_type", inplace=True)

In [5]:
# set target
y = df["aae_realization"]
X = df.drop(columns="aae_realization")

In [6]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=seed
)

In [7]:
# normalize
normalizer = StandardScaler()
X_train["year_norm"] = normalizer.fit_transform(X_train[["year"]])
X_test["year_norm"] = normalizer.transform(X_test[["year"]])

In [8]:
X_train_cat = X_train.drop(columns=["year", "year_norm"])
X_train_norm = X_train.drop(columns=["year", "time"])
X_train_year = X_train.drop(columns=["time", "year_norm"])

X_test_cat = X_test.drop(columns=["year", "year_norm"])
X_test_norm = X_test.drop(columns=["year", "time"])
X_test_year = X_test.drop(columns=["time", "year_norm"])

cat_features_cat = list(X_train_cat.select_dtypes("object").columns)
cat_features_norm = list(X_train_norm.select_dtypes("object").columns)
cat_features_year = list(X_train_year.select_dtypes("object").columns)

In [9]:
# cat
model = catboost.CatBoostClassifier()
model.fit(
    X_train_cat,
    y_train,
    cat_features=cat_features_cat,
    verbose=0,
    early_stopping_rounds=100,
)

preds = model.predict(X_test_cat)
score = f1_score(y_test, preds)
print("categorical year f1 score:", score)

categorical year f1 score: 0.9730816486094047


In [10]:
# norm
model = catboost.CatBoostClassifier()
model.fit(
    X_train_norm,
    y_train,
    cat_features=cat_features_norm,
    verbose=0,
    early_stopping_rounds=100,
)

preds = model.predict(X_test_norm)
score = f1_score(y_test, preds)
print("normalized year f1 score:", score)

normalized year f1 score: 0.9726103968697597


In [11]:
# year
model = catboost.CatBoostClassifier()
model.fit(
    X_train_year,
    y_train,
    cat_features=cat_features_year,
    verbose=0,
    early_stopping_rounds=100,
)

preds = model.predict(X_test_year)
score = f1_score(y_test, preds)
print("regular year f1 score:", score)

regular year f1 score: 0.9726103968697597


categorical feature results in best model performance